<a href="https://colab.research.google.com/github/bhatiatanish225/Resnet50Model/blob/main/Resnet50Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
import time

import tensorflow as tf
from tensorflow import keras
from tensorflow.python.compiler.tensorrt import trt_convert as trt
from tensorflow.python.saved_model import tag_constants
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions


In [4]:
model = ResNet50(weights='imagenet')
model.export('resnet50_saved_model')

Saved artifact at 'resnet50_saved_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_177')
Output Type:
  TensorSpec(shape=(None, 1000), dtype=tf.float32, name=None)
Captures:
  133819859468752: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133816013384464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133816013384848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133816013384080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133816013383120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133816013384656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133816013387920: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133816013389072: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133816013383888: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133816013389456: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133

In [6]:
import numpy as np

input_saved_model_dir='/content/resnet50_saved_model'
print('Loading saved model {}...', format(input_saved_model_dir))
saved_model_loaded = tf.saved_model.load(input_saved_model_dir,tags=[tag_constants.SERVING])
infer = saved_model_loaded.signatures['serving_default']
BATCH_SIZE = 32
dummy_input_batch = np.zeros((BATCH_SIZE,224,224,3),dtype=np.float32)
dummy_input_batch =tf.constant(dummy_input_batch)

Loading saved model {}... /content/resnet50_saved_model


In [7]:

def predict_and_benchmark_throughput_from_saved(batched_input, infer, N_warmup_run=50, N_run=500, model='resnet_v2'):
    elapsed_time = []
    all_preds = []
    batch_size = batched_input.shape[0]

    label_key = None

    if model == 'resnet_v2':
        label_key = 'probs'
    elif model in ['vgg19', 'inception_v3']:
        label_key = 'predictions'
    else:
        raise ValueError("Unsupported model name: %s" % model)

    for i in range(N_warmup_run):
        labeling = infer(batched_input)
#         preds = labeling[label_key].numpy()


    for i in range(N_run):
        start_time = time.time()

        # This call to infer is asynchronous, therefore, for the sake of benchmarking...
        labeling = infer(batched_input)

        # ...we demand a return value to force the execution to wait, before ending the timer
#         preds = labeling[label_key].numpy()

        end_time = time.time()
        elapsed_time = np.append(elapsed_time, end_time - start_time)

#         all_preds.append(preds)

        if i % 50 == 0:
            print('Steps {}-{} average: {:4.1f}ms'.format(i, i+50, (elapsed_time[-50:].mean()) * 1000))

    throughput = N_run * batch_size / elapsed_time.sum()
    print('Throughput: {:.0f} images/s'.format(throughput))
    return all_preds, throughput

In [8]:
predict_and_benchmark_throughput_from_saved(dummy_input_batch,infer,N_warmup_run=5,N_run=500 , model='resnet_v2')

Steps 0-50 average: 109.5ms
Steps 50-100 average: 113.0ms
Steps 100-150 average: 110.9ms
Steps 150-200 average: 111.1ms
Steps 200-250 average: 111.5ms
Steps 250-300 average: 111.7ms
Steps 300-350 average: 112.4ms
Steps 350-400 average: 113.1ms
Steps 400-450 average: 113.4ms
Steps 450-500 average: 113.9ms
Throughput: 284 images/s


([], 284.42564384496154)

In [11]:
# conversion_params = trt.DEFAULT_TRT_CONVERSION_PARAMS._replace(precision_model=trt.TrtPrecisionMode.FP16 , max_workspace_size_bytes=8000000000)
# converter = trt.TrtGraphConverterV2(input_saved_model_dir='models/simple-tensorflow-model/1/model.savedmodel')

# converter.convert()
# converter.save(output_saved_model_dir='models/simple-tensorflow-model/2/model.savedmodel')
# print('Done Converting to Tf-TRT FP16')